<a href="https://colab.research.google.com/github/acdc2019/algo-trading/blob/main/python/notebooks/strategies/rsi_strategy/RSIStrategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **RSI Strategy**
## **15 mins strategy**
* Step 1: A 15 min candle has to close above 70 RSI value.
* Step 2: Above High of this candle Buy signal is generated.
* Step 3: Stop Loss of the trade has to be below the earlier 15 min candles low.
* Step 4: Volume of the candle in which buy signal is generated has to be atleast 2 to 3 times of earlier 5 candles (***Q: average of earlier 5 candle volume?***)
* Step 5: Hourly candles RSI should be above 50 for confirmation.
* Step 6: In the last 5 candles the stock movement should not be more than 4% of the day movement
* Step 7: If Stop Loss of the candle is less than 6000 INR then the trade should be executed

#### **Strategy Parameters**
* window_start, window_end: Dates between which to look for signal
* rsi_15min = 70
* rsi_60min = 50
* stop_loss = 6000
* back_candles = 5
* volume_multiple = 2 or 3 times
* daily_movement_pct = 4

Install libs

In [1]:
!pip install ta
!pip install pandas==1.3.5
!pip install plotly
!pip install numpy

  Created wheel for ta: filename=ta-0.9.0-py3-none-any.whl size=28908 sha256=bbb2b20c458d12616e33d4bd1104afcbfccf2f7f0d71508aaa359d29ff1c5cec
  Stored in directory: /root/.cache/pip/wheels/72/78/64/cc1c01506a1010a9845e9bd7c69333730f7174661228ea4f98
Successfully built ta
     |████████████████████████████████| 11.3 MB 4.2 MB/s 
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas~=1.1.0; python_version >= "3.0", but you have pandas 1.3.5 which is incompatible.


In [1]:
!wget https://raw.githubusercontent.com/acdc2019/algo-trading/main/python/strategy/helpers.py?token=GHSAT0AAAAAABQZJYETEQYUGLNGHAJZQ7CAYPP566Q -O helpers.py

--2022-01-25 12:39:39--  https://raw.githubusercontent.com/acdc2019/algo-trading/main/python/strategy/helpers.py?token=GHSAT0AAAAAABQZJYETEQYUGLNGHAJZQ7CAYPP566Q
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... ^C


Import Libs

In [142]:
import pandas as pd
from ta.momentum import RSIIndicator
from ta.trend import ADXIndicator
from plotly.subplots import make_subplots
from datetime import date
import numpy as np

In [7]:
class Signal():
    def __init__(self, strategy: str, sym: str, lot_size: int, ts: date, entry_price: float, stop_loss: float,
                 rsi: float, hourly_rsi: float, vol: int, mean_vol: int, daily_mov_pct: float) -> None:
        self.strategy = strategy
        self.sym = sym
        self.lot_size = lot_size
        self.ts = ts
        self.entry_price = entry_price
        self.stop_loss = stop_loss
        self.rsi = rsi
        self.hourly_rsi = hourly_rsi
        self.vol = vol
        self.mean_vol = mean_vol
        self.daily_mov_pct = daily_mov_pct
        self.pnl = 0
        self.comment = ""

    def __str__(self) -> str:
        return "Strategy: {}, Sym: {}, TS: {}, Entry: {}, StopLoss: {}, PnL: {}, Comment: {}".format(
            self.strategy, self.sym, self.ts, self.entry_price, self.stop_loss, self.pnl, self.comment)

class BackTestResult():
    def __init__(self, sym: str, signals: list) -> None:
        self.signals = signals
        self.sym = sym
        self.total_pnl = self._calc_total_pnl()
        self.gross_profit = self._calc_gross_profit()
        self.gross_loss = self._calc_gross_loss()
        self.profit_factor = self._calc_profit_factor()

    def _calc_profit_factor(self) -> float:
        gross_profit = self._calc_gross_profit()
        gross_loss = abs(self._calc_gross_loss())
        if (gross_loss == 0):
            gross_loss = 0.1

        profit_factor = gross_profit/gross_loss
        return round(profit_factor, 2)

    def _calc_gross_profit(self) -> float:
        gross_profit = 0
        for signal in self.signals:
            if (signal.pnl > 0):
                gross_profit = gross_profit + signal.pnl

        return round(gross_profit, 2)

    def _calc_gross_loss(self) -> float:
        gross_loss = 0
        for signal in self.signals:
            if (signal.pnl < 0):
                gross_loss = gross_loss - signal.pnl

        return round(gross_loss, 2)

    def _calc_strike_rate(self) -> float:
        if (len(self.signals) == 0):
            return 0

        profit = 0
        notional = 0
        for signal in self.signals:
            if (signal.pnl > 0):
                profit = profit+signal.pnl

            notional = notional + abs(signal.pnl)

        return round(profit/notional, 2)

    def _calc_total_pnl(self) -> float:
        total_pnl = 0
        for signal in self.signals:
            total_pnl = total_pnl + signal.pnl

        return round(total_pnl, 2)

    def __str__(self) -> str:
        return 'Sym,{},Total PnL,{},ProfitFactor,{},GrossProfit,{},GrossLoss,{},Total Signals,{}'.format(
            self.sym, self.total_pnl, self.profit_factor, self.gross_profit, self.gross_loss, len(self.signals))


In [108]:
def get_previous_candles(df: pd.DataFrame, index, n: int, include_index=False):
    '''
    Returns previous n candles from the given index in the DataFrame
    Parameters:
    df (DataFrame): DataFrame from which to return the previous candles
    index (DataFrame Index): DataFrame Index from which to return the previous candles
    n (int): Number of previous candles to return from index
    include_index (bool): If current index should be included in returned DataFrame
    Returns:
    DataFrame: Pandas dataframe with the previous n candles
    '''
    loc = df.index.get_loc(index)
    fromIdx = loc-n
    toIdx = loc+1 if include_index else loc
    return df.iloc[fromIdx:toIdx]


def get_next_candles(df: pd.DataFrame, index, n: int):
    '''
    Returns next n candles from the given index in the DataFrame
    Parameters:
    df (DataFrame): DataFrame from which to return the next candles
    index (DataFrame Index): DataFrame Index from which to return the next candles
    n (int): Number of next candles to return from index
    Returns:
    DataFrame: Pandas dataframe with the next n candles
    '''
    loc = df.index.get_loc(index)
    return df.iloc[loc+1:loc+1+n]


def get_hourly_df(df: pd.DataFrame, index) -> pd.DataFrame:
    df_temp = df[:index]
    # print(index)
    # print(df_15min_temp.tail(10))
    df_60min_o = df_temp['Open'].resample(
        '60Min', offset='30Min').apply({'Open': 'first'})
    df_60min_h = df_temp['High'].resample(
        '60Min', offset='30Min').apply({'High': 'max'})
    df_60min_l = df_temp['Low'].resample(
        '60Min', offset='30Min').apply({'Low': 'min'})
    df_60min_c = df_temp['Close'].resample(
        '60Min', offset='30Min').apply({'Close': 'last'})
    df_60min_vol = df_temp['Volume'].resample(
        '60Min', offset='30Min').apply({'Volume': 'sum'})

    df_60min = pd.concat([df_60min_o, df_60min_h, df_60min_l,
                         df_60min_c, df_60min_vol], axis=1)
    df_60min.dropna(subset=['Open'], inplace=True)

    rsi = RSIIndicator(df_60min['Close']).rsi()
    df_60min = df_60min.assign(rsi=rsi.values)
    return df_60min


def get_tanaji_pct(df: pd.DataFrame, index, n: int, high: float) -> float:
    prev_candles = get_previous_candles(df, index, n)
    min_low = prev_candles['Low'].min()
    daily_movement = high - min_low
    tanaji_pct = (daily_movement/min_low)*100
    return tanaji_pct


def get_sell_tanaji_pct(df: pd.DataFrame, index, n: int, low: float) -> float:
    prev_candles = get_previous_candles(df, index, n)
    max_high = prev_candles['High'].max()
    daily_movement = max_high - low
    tanaji_pct = (daily_movement/max_high)*100
    return tanaji_pct


def generate_buy_signal_pnl(signal: Signal, df: pd.DataFrame, stop_gain: int) -> None:
    df = df[signal.ts:].copy()
    df.drop(index=df.index[0], axis=0, inplace=True)
    rnum = 0
    stop_loss_gap = signal.entry_price - signal.stop_loss
    rsi_breached = False

    for index, row in df.iterrows():
        if (row['Low'] < signal.stop_loss):
            #print('rnum: {}, index: {}, low: {}, sl: {}'.format(rnum, index, row['Low'], signal.stop_loss))
            signal.pnl = -1 * (signal.entry_price -
                               signal.stop_loss) * signal.lot_size
            signal.comment = "Stop Loss Breached at {}".format(index)
            break

        if ((row['High'] - signal.entry_price) * signal.lot_size >= stop_gain):
            signal.pnl = stop_gain
            signal.comment = "Stop Gain reached at {}".format(index)
            break

        if (rsi_breached == True):
            signal.pnl = (row['Open'] - signal.entry_price) * signal.lot_size
            signal.comment = "RSI Breached at {}".format(index)
            break

        if (row['High'] - signal.stop_loss > 2*stop_loss_gap):
            # print('New Stoploss: {}, Old Stoploss: {}, Stoploss gap: {}'.format(                row['High'] - stop_loss_gap, signal.stop_loss, stop_loss_gap))
            # signal.stop_loss = row['High'] - stop_loss_gap
            signal.stop_loss = signal.stop_loss

        '''if (row['rsi'] < 70 and row['Volume'] >= 2*row['vol_sma5']):
            rsi_breached = True'''

        if(rnum > 50):
            signal.pnl = (row['Close'] -
                          signal.entry_price) * signal.lot_size
            # print('Signal failed to generate PnL: {}'.format(signal.pnl))
            signal.comment = "Signal failed to generate {}".format(index)
            break

        rnum = rnum+1


def generate_sell_signal_pnl(signal: Signal, df: pd.DataFrame, stop_gain: float) -> None:
    df = df[signal.ts:].copy()
    df.drop(index=df.index[0], axis=0, inplace=True)
    rnum = 0
    for index, row in df.iterrows():
        if (row['High'] > signal.stop_loss):
            #print('rnum: {}, index: {}, low: {}, sl: {}'.format(rnum, index, row['Low'], signal.stop_loss))
            signal.pnl = (signal.entry_price -
                          signal.stop_loss) * signal.lot_size
            break

        if ((signal.entry_price - row['Low']) * signal.lot_size >= stop_gain):
            signal.pnl = stop_gain
            break

        if(rnum > 50):
            signal.pnl = (signal.entry_price - row['Close']) * signal.lot_size
            # print('Signal failed to generate PnL: {}'.format(signal.pnl))
            break
        rnum = rnum+1

## Load 15 min and 60 min stock data and calculate RSI

In [162]:
file_15min = '/content/RELIANCE-HIST.csv'

df_15min = pd.read_csv(file_15min, parse_dates=['Date'], index_col=['Date'])

# Add RSI
rsi = RSIIndicator(df_15min['Close']).rsi()
df_15min = df_15min.assign(rsi=rsi.values)

# Add Vol SMA 5
vol_sma5 = df_15min['Volume'].rolling(5).mean()
df_15min = df_15min.assign(vol_sma5=vol_sma5.values)

# Add ADX
ADX = ADXIndicator(high=df_15min['High'], low=df_15min['Low'], close=df_15min['Close'])
df_15min = df_15min.assign(adx=ADX.adx().values)
df_15min = df_15min.assign(adx_neg=ADX.adx_neg().values)
df_15min = df_15min.assign(adx_pos=ADX.adx_pos().values)
df_15min.head(50)


/usr/local/lib/python3.7/dist-packages/ta/trend.py:769: RuntimeWarning:

invalid value encountered in double_scalars

/usr/local/lib/python3.7/dist-packages/ta/trend.py:774: RuntimeWarning:

invalid value encountered in double_scalars



,Open,High,Low,Close,Volume,rsi,vol_sma5,adx,adx_neg,adx_pos
Date,,,,,,,,,,
2015-02-02 09:15:00+05:30,454.15,455.60,452.75,453.95,593013,NaN,NaN,0.000000,0.000000,0.000000
2015-02-02 09:30:00+05:30,454.05,454.15,452.70,452.75,291580,NaN,NaN,0.000000,0.000000,0.000000
2015-02-02 09:45:00+05:30,452.75,453.60,452.15,452.95,234579,NaN,NaN,0.000000,0.000000,0.000000
2015-02-02 10:00:00+05:30,452.95,453.45,451.45,452.70,210231,NaN,NaN,0.000000,0.000000,0.000000
2015-02-02 10:15:00+05:30,452.70,453.20,452.20,452.70,91731,NaN,284226.8,0.000000,0.000000,0.000000
2015-02-02 10:30:00+05:30,452.75,452.95,452.30,452.65,81630,NaN,181950.2,0.000000,0.000000,0.000000
2015-02-02 10:45:00+05:30,452.65,452.95,452.25,452.65,138169,NaN,151268.0,0.000000,0.000000,0.000000
2015-02-02 11:00:00+05:30,452.65,452.70,451.70,451.70,82045,NaN,120761.2,0.000000,0.000000,0.000000
2015-02-02 11:15:00+05:30,451.70,452.30,451.50,452.20,82655,NaN,95246.0,0.000000,0.000000,0.000000


## Set Strategy Parameters

In [163]:
# Strategy params
strategy = 'RSIBuy'
sym = 'RELIANCE'
window_start = '2021-01-01 00:00:00'
window_end = '2021-12-15 00:00:00'
rsi_15min = 70
rsi_60min = 50
lot_size = 250
stop_loss = 6500
stop_gain = 25000
back_candles = 5
volume_multiple = 2  # 2 or 3 times
daily_movement_pct = 4
check_volume = True


## Actual Strategy Implementation

### Step 1: A 15 min candle has to close above 70 RSI value
Get all the candles in the window that close above required RSI value

In [164]:
curr_window_df = df_15min[window_start:window_end]

rsi_filter = (curr_window_df['rsi'] > rsi_15min)
df = curr_window_df[rsi_filter]

bullish_filter = (df['Close'] > df['Open'])
df = df[bullish_filter]

temp_df = pd.DataFrame(columns=df_15min.columns)
temp_df.index.name = 'Date'
back_candles_df = pd.DataFrame()

for index, row in df.iterrows():
    back_candles_df = get_previous_candles(curr_window_df, index, 1)
    prev_rsi = 0
    if (back_candles_df.shape[0] != 0):
        prev_rsi = back_candles_df.iloc[0]['rsi']

    if (prev_rsi < 70):
        temp_df.loc[index] = row

df = temp_df[df_15min.columns]
df.head()


,Open,High,Low,Close,Volume,rsi,vol_sma5,adx,adx_neg,adx_pos
Date,,,,,,,,,,
2021-01-12 13:00:00+05:30,1930.30,1939.95,1930.20,1939.95,700503.0,71.475675,402157.8,25.849872,16.137759,34.720110
2021-01-18 14:00:00+05:30,1979.85,1993.75,1977.70,1991.20,1449744.0,73.754175,719728.4,31.704905,9.768242,35.386654
2021-01-18 14:30:00+05:30,1987.05,1992.55,1986.90,1990.00,746061.0,70.681392,958880.0,35.183491,8.798959,32.282477
2021-01-19 09:15:00+05:30,1994.65,2018.95,1994.65,2014.05,2633107.0,70.765099,1308107.0,36.888278,14.528444,38.616944
2021-01-19 12:00:00+05:30,2023.00,2029.10,2022.90,2028.65,510241.0,71.707619,426415.4,37.754890,16.057625,34.079643



### Step 4: Volume check

In [165]:
if (check_volume == True):
    temp_df = pd.DataFrame(columns=df_15min.columns)
    temp_df.index.name = 'Date'

    for index, row in df.iterrows():
        # Compare volume against previous mean volume
        if(row['Volume'] > volume_multiple*row['vol_sma5']):
            temp_df.loc[index] = row

    df = temp_df[df_15min.columns]
    df.head()

### Step 5: Hourly candle RSI check

In [166]:
temp_df = pd.DataFrame(columns=df.columns)
temp_df.index.name = 'Date'

for index, row in df.iterrows():
    df_60min = get_hourly_df(df_15min, index)
    # print(df_60min.tail(15))

    rsi = df_60min.iloc[-1]['rsi']
    if(rsi > rsi_60min):
        # This 15min candle is eligible for signal
        # print('Hourly candle RSI is greater than 50..', rsi)
        temp_df.loc[index] = row

df = temp_df
df.head()


,Open,High,Low,Close,Volume,rsi,vol_sma5,adx,adx_neg,adx_pos
Date,,,,,,,,,,
2021-01-18 14:00:00+05:30,1979.85,1993.75,1977.70,1991.20,1449744.0,73.754175,719728.4,31.704905,9.768242,35.386654
2021-01-19 09:15:00+05:30,1994.65,2018.95,1994.65,2014.05,2633107.0,70.765099,1308107.0,36.888278,14.528444,38.616944
2021-01-21 09:15:00+05:30,2082.00,2090.00,2075.55,2083.60,2278410.0,80.684406,1113439.0,39.879363,6.231189,45.645333
2021-02-02 14:30:00+05:30,1883.10,1910.00,1883.00,1909.35,1539153.0,71.795337,720888.8,17.031168,14.170319,40.257542
2021-02-09 09:30:00+05:30,1970.15,1989.55,1969.35,1982.00,1502636.0,70.708793,731307.6,38.561569,12.704095,41.839559


### Step 6: Stock movement check

In [167]:
temp_df = pd.DataFrame(columns=df.columns)
temp_df.index.name = 'Date'

for index, row in df.iterrows():
    tanaji_pct = get_tanaji_pct(df_15min, index, 5, row['High'])
    # print('tanaji_pct:', tanaji_pct)
    if(tanaji_pct < daily_movement_pct):
        temp_df.loc[index] = row

df = temp_df
df.head()

,Open,High,Low,Close,Volume,rsi,vol_sma5,adx,adx_neg,adx_pos
Date,,,,,,,,,,
2021-01-18 14:00:00+05:30,1979.85,1993.75,1977.70,1991.20,1449744.0,73.754175,719728.4,31.704905,9.768242,35.386654
2021-01-19 09:15:00+05:30,1994.65,2018.95,1994.65,2014.05,2633107.0,70.765099,1308107.0,36.888278,14.528444,38.616944
2021-01-21 09:15:00+05:30,2082.00,2090.00,2075.55,2083.60,2278410.0,80.684406,1113439.0,39.879363,6.231189,45.645333
2021-02-02 14:30:00+05:30,1883.10,1910.00,1883.00,1909.35,1539153.0,71.795337,720888.8,17.031168,14.170319,40.257542
2021-02-09 09:30:00+05:30,1970.15,1989.55,1969.35,1982.00,1502636.0,70.708793,731307.6,38.561569,12.704095,41.839559


### Step 7: Stop Loss check

In [168]:
temp_df = pd.DataFrame(columns=df.columns)
temp_df.index.name = 'Date'

for index, row in df.iterrows():
    prev_candle = get_previous_candles(df_15min, index, 1)
    prev_low = prev_candle.iloc[0]['Low']
    if((row['High'] - prev_low)*lot_size > stop_loss):
        print(index, 'Stop Loss greater than 6000 INR. Do not trade', (row['High'] - prev_low)*lot_size)
        continue
    else:
        print(index, 'Stop Loss within range', (row['High'] - prev_low)*lot_size)
        temp_df.loc[index] = row

df = temp_df
df

2021-01-18 14:00:00+05:30 Stop Loss within range 5500.0
2021-01-19 09:15:00+05:30 Stop Loss greater than 6000 INR. Do not trade 11737.500000000011
2021-01-21 09:15:00+05:30 Stop Loss greater than 6000 INR. Do not trade 9375.0
2021-02-02 14:30:00+05:30 Stop Loss greater than 6000 INR. Do not trade 8949.999999999989
2021-02-09 09:30:00+05:30 Stop Loss greater than 6000 INR. Do not trade 7349.999999999965
2021-02-25 09:15:00+05:30 Stop Loss greater than 6000 INR. Do not trade 9500.0
2021-02-25 12:45:00+05:30 Stop Loss within range 4787.500000000023
2021-03-03 14:15:00+05:30 Stop Loss greater than 6000 INR. Do not trade 8825.000000000045
2021-03-23 10:30:00+05:30 Stop Loss greater than 6000 INR. Do not trade 11625.0
2021-04-07 09:15:00+05:30 Stop Loss greater than 6000 INR. Do not trade 6850.000000000023
2021-04-15 15:00:00+05:30 Stop Loss within range 4774.999999999977
2021-04-27 09:15:00+05:30 Stop Loss greater than 6000 INR. Do not trade 10600.000000000022
2021-04-27 11:30:00+05:30 Stop

,Open,High,Low,Close,Volume,rsi,vol_sma5,adx,adx_neg,adx_pos
Date,,,,,,,,,,
2021-01-18 14:00:00+05:30,1979.85,1993.75,1977.70,1991.20,1449744.0,73.754175,719728.4,31.704905,9.768242,35.386654
2021-02-25 12:45:00+05:30,2095.35,2108.40,2095.00,2107.55,857631.0,73.523040,328238.8,34.441396,12.171959,37.325597
2021-04-15 15:00:00+05:30,1938.05,1946.00,1937.70,1944.00,860414.0,70.635080,384199.8,20.249750,11.602793,35.857612
2021-04-27 11:30:00+05:30,1970.50,1984.50,1970.50,1980.80,729633.0,75.821669,241883.6,33.015066,13.910341,42.748820
2021-05-17 14:30:00+05:30,1953.00,1963.45,1952.35,1961.20,572911.0,74.875418,185648.0,47.406131,6.781973,33.272523
2021-06-14 13:45:00+05:30,2241.65,2255.00,2239.95,2244.15,1101967.0,70.489738,503501.2,19.204735,11.966053,35.586670
2021-06-22 09:15:00+05:30,2254.00,2259.00,2244.80,2254.10,901076.0,70.739613,426544.4,23.427684,14.757044,37.342187
2021-07-16 10:30:00+05:30,2094.45,2109.00,2094.20,2108.65,313269.0,71.785837,152553.8,17.513584,18.758866,42.447396
2021-08-03 15:15:00+05:30,2085.65,2091.00,2085.00,2090.45,938265.0,71.900984,392875.2,37.914680,9.290498,33.013213


## **Final Signal**

In [169]:
signals = list()
for index, row in df.iterrows():
    stop_loss_candle = get_previous_candles(df_15min, index, 1)
    sig_stop_loss = stop_loss_candle.iloc[0]['Low']
    sig_entry = row['High']
    sig_daily_mov_pct = np.round(get_tanaji_pct(df_15min, index, 5, row['High']), 2)
    sig_rsi = np.round(row['rsi'], 2)
    sig_hourly_rsi = np.round(get_hourly_df(df_15min, index).iloc[-1]['rsi'], 2)
    sig_mean_volume = row['vol_sma5']
    sig_volume = row['Volume']
    signal = Signal(strategy=strategy, sym=sym, lot_size=lot_size,
                    ts=index, entry_price=sig_entry, stop_loss=sig_stop_loss,
                    rsi=sig_rsi, hourly_rsi=sig_hourly_rsi, vol=sig_volume,
                    mean_vol=sig_mean_volume, daily_mov_pct=sig_daily_mov_pct)
    signals.append(signal)

for signal in signals:
    generate_buy_signal_pnl(signal, df_15min, stop_gain=stop_gain)

btResult = BackTestResult(sym=sym, signals=signals)

for signal in btResult.signals:
    print(signal)

print(btResult)

Strategy: RSIBuy, Sym: RELIANCE, TS: 2021-01-18 14:00:00+05:30, Entry: 1993.75, StopLoss: 1971.75, PnL: 13799.999999999955, Comment: Signal failed to generate 2021-01-20 14:30:00+05:30
Strategy: RSIBuy, Sym: RELIANCE, TS: 2021-02-25 12:45:00+05:30, Entry: 2108.4, StopLoss: 2089.25, PnL: -4787.500000000023, Comment: Stop Loss Breached at 2021-02-26 11:15:00+05:30
Strategy: RSIBuy, Sym: RELIANCE, TS: 2021-04-15 15:00:00+05:30, Entry: 1946.0, StopLoss: 1926.9, PnL: -4774.999999999977, Comment: Stop Loss Breached at 2021-04-16 15:15:00+05:30
Strategy: RSIBuy, Sym: RELIANCE, TS: 2021-04-27 11:30:00+05:30, Entry: 1984.5, StopLoss: 1969.2, PnL: 8375.0, Comment: Signal failed to generate 2021-04-29 12:00:00+05:30
Strategy: RSIBuy, Sym: RELIANCE, TS: 2021-05-17 14:30:00+05:30, Entry: 1963.45, StopLoss: 1951.25, PnL: 8237.500000000011, Comment: Signal failed to generate 2021-05-19 15:00:00+05:30
Strategy: RSIBuy, Sym: RELIANCE, TS: 2021-06-14 13:45:00+05:30, Entry: 2255.0, StopLoss: 2230.45, PnL

In [170]:
import plotly.graph_objects as go
import plotly.offline as py

back=5
next=50
sig_param_col1 = ['Time','Entry INR','StopLoss INR','Daily Mov %','Volume','Mean Vol','15Min RSI','Hourly RSI','PnL','Comments']
for signal in signals:
  prev_candles = get_previous_candles(df_15min, signal.ts, back, True)
  next_candles = get_next_candles(df_15min, signal.ts, next)
  
  candles = pd.concat([prev_candles, next_candles])
  candles['DateStr'] = candles.index.strftime('%d-%m %H:%M')

  # Get all strategy params for this signal

  sig_stop_loss = signal.stop_loss
  sig_entry = signal.entry_price
  sig_daily_mov_pct = signal.daily_mov_pct
  sig_rsi = signal.rsi
  sig_hourly_rsi = signal.hourly_rsi
  sig_mean_volume = signal.mean_vol
  sig_volume = signal.vol

  fig = make_subplots(rows=3, cols=2, shared_xaxes=False,
               subplot_titles=('OHLC', 'Signal Params', 'Volume & RSI','', 'ADX', ''), 
               vertical_spacing=0.1, 
               horizontal_spacing=0.01,
               row_width=[0.25, 0.25, 0.5],
               column_widths=[0.8,0.2],
               specs=[[{"secondary_y": False, "type": "candlestick"},{"secondary_y": False, "type":"table"}], 
                      [{"secondary_y": True}, {"secondary_y": False}],
                      [{"secondary_y": True}, {"secondary_y": False}]])
    
  fig.add_trace(go.Candlestick(x=candles['DateStr'],
                     open=candles['Open'],
                     high=candles['High'],
                     low=candles['Low'],
                     close=candles['Close'],
                     name='Signal Chart',
                     increasing_line_color='yellow',
                     increasing_fillcolor='yellow',
                     decreasing_line_color='red',
                     decreasing_fillcolor='red',),
                     row=1,col=1)
  
  fig.add_annotation(x=back,y=sig_entry,
                     text='Signal')
  
  # Position Entry Point
  fig.add_shape(type='line', 
                x0=-1,x1=back+next+2,
                y0=sig_entry, y1=sig_entry, 
                line=dict(color='Green'),
                row=1,col=1)
  
  # Position Stop Loss
  fig.add_shape(type='line', 
                x0=-1,x1=back+next+2,
                y0=sig_stop_loss, y1=sig_stop_loss, 
                line=dict(color='Red'),
                row=1,col=1)
  
  fig.add_annotation(x=back,y=sig_rsi,
                      text='Signal',row=2, col=1)
  
  fig.add_shape(type='line', x0=-1,x1=back+next+2,y0=rsi_15min, y1=rsi_15min, 
                line=dict(color='Green'),
                row=2,col=1)

  # Signal Parameters Table
  fig.add_trace(go.Table(header=dict(values=['Param','Value'],
                                     line_color='white',
                                     fill_color='darkslategray',
                                     align='left'),
                         cells=dict(values=[sig_param_col1,
                                           [signal.ts.strftime('%d-%m %H:%M'), sig_entry, 
                                            sig_stop_loss, sig_daily_mov_pct, sig_volume, 
                                            sig_mean_volume, sig_rsi, sig_hourly_rsi, signal.pnl, signal.comment]],
                                    line_color='white',
                                    fill_color='black',
                                    align='left')),
                row=1, col=2)

  # RSI
  fig.add_trace(go.Scatter(x=candles['DateStr'], y=candles['rsi'], name='rsi',            
           marker_color='Cyan'),
           row=2, col=1)
  
  # Volume
  fig.add_trace(go.Bar(x=candles['DateStr'], y=candles['Volume'], name='Volume',            
           marker_color='rgb(55, 55, 109)',
           width=np.array([0.5]*candles.size)),
           secondary_y=True,
          row=2, col=1)
  
  # ADX
  fig.add_trace(go.Scatter(x=candles['DateStr'], y=candles['adx'], name='adx',            
           marker_color='Cyan'),
           row=3, col=1)
  fig.add_trace(go.Scatter(x=candles['DateStr'], y=candles['adx_neg'], name='adx_neg',            
           marker_color='Blue'),
           row=3, col=1)
  fig.add_trace(go.Scatter(x=candles['DateStr'], y=candles['adx_pos'], name='adx_pos',            
           marker_color='Yellow'),
           row=3, col=1)

  fig.update_xaxes(type='category', rangeslider=dict(visible=False))
  fig.update_xaxes(showgrid=False, nticks=5)
  fig.update_yaxes(showgrid=False)
  fig.update_layout(
    title='Signal generated for RSI 15mins Strategy',
    title_x = 0.5,
    autosize=False,
    width=1450,
    height=850,
    plot_bgcolor='rgb(5,5,5)',
    paper_bgcolor='rgb(0,0,0)',
    font_color='white')

  py.iplot(fig)
  